In [9]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats.contingency import crosstab

In [21]:
df = pd.read_csv("survey_results_public.csv")

In [22]:
print(df['Hobbyist'].isna().sum())            # relatively few nulls (45)
df = df.dropna(how='any', subset='Hobbyist')  # drop all rows in the response var col without values

# categorical columns
cat_vars = df.drop(['Respondent', 
                    'Age', 
                    'Age1stCode',
                    'CompTotal', 
                    'ConvertedComp',
                    'WorkWeekHrs',
                    'YearsCode',
                    'YearsCodePro'], 
                    axis=1)

# conversion to make these a quanititative var
df['YearsCode'].replace('Less than 1 year', 0.5, inplace=True) 
df['YearsCode'].replace('More than 50 years', 55, inplace=True)  # it's unlikely they coded for more than 60 years
df['YearsCodePro'].replace('Less than 1 year', 0.5, inplace=True) 
df['YearsCodePro'].replace('More than 50 years', 55, inplace=True)

# true values unlikely to be far from these replacements
df['Age1stCode'].replace('Younger than 5 years', 3, inplace=True)  
df['Age1stCode'].replace('Older than 85', 87, inplace=True)  

# quantitative columns
quant_vars = df.filter(['Respondent',
                        'Age',
                        'Age1stCode',
                        'CompTotal',
                        'ConvertedComp',
                        'WorkWeekHrs',
                        'YearsCode',
                        'YearsCodePro'],
                        axis=1)

# for some reason, the data is all of type 'object', not strings or floats, so these convert it
cat_vars = pd.DataFrame(cat_vars, dtype=str)  
quant_vars = pd.DataFrame(quant_vars, dtype=float)

45
               Respondent    Age  Age1stCode  CompTotal  ConvertedComp  \
Respondent           True  False       False      False          False   
Age                 False   True       False      False          False   
Age1stCode          False  False        True      False          False   
CompTotal           False  False       False      False          False   
ConvertedComp       False  False       False      False           True   
WorkWeekHrs         False  False       False      False          False   
YearsCode           False   True       False      False          False   
YearsCodePro        False   True       False      False          False   

               WorkWeekHrs  YearsCode  YearsCodePro  
Respondent           False      False         False  
Age                  False       True          True  
Age1stCode           False      False         False  
CompTotal            False      False         False  
ConvertedComp        False      False         False  
WorkWe

To slim down the features further, we can exclude categorical features with very high numbers of levels. So let's see the number of unique values for each categorical feature:

In [12]:
# count number of levels on each categorical column, drop ones with high number of levels
# sorted for convenience
col_unique_counts = cat_vars.nunique(dropna=True).sort_values(axis=0, ascending=True)
print(col_unique_counts)

Hobbyist                            2
Trans                               3
NEWOtherComms                       3
NEWDevOps                           4
CompFreq                            4
SurveyLength                        4
SurveyEase                          4
NEWOffTopic                         4
NEWOnboardGood                      4
SOAccount                           4
PurchaseWhat                        4
JobSeek                             4
NEWLearn                            5
OpSys                               5
NEWPurpleLink                       5
JobSat                              6
NEWDevOpsImpt                       6
NEWEdImpt                           6
NEWOvertime                         6
MainBranch                          6
SOComm                              7
SOPartFreq                          7
SOVisitFreq                         7
WelcomeChange                       7
Gender                              8
Employment                          8
EdLevel     

Some of these variables have hundreds or eve thousands of levels! Looks like we can drop about everything after `Sexuality` (and even keeping that may be a bit much).

In [13]:
small_cats = col_unique_counts.loc[:'Sexuality'] # categorical variables with smaller number of levels
cat_vars = cat_vars[small_cats.index]
cat_vars.columns

Index(['Hobbyist', 'Trans', 'NEWOtherComms', 'NEWDevOps', 'CompFreq',
       'SurveyLength', 'SurveyEase', 'NEWOffTopic', 'NEWOnboardGood',
       'SOAccount', 'PurchaseWhat', 'JobSeek', 'NEWLearn', 'OpSys',
       'NEWPurpleLink', 'JobSat', 'NEWDevOpsImpt', 'NEWEdImpt', 'NEWOvertime',
       'MainBranch', 'SOComm', 'SOPartFreq', 'SOVisitFreq', 'WelcomeChange',
       'Gender', 'Employment', 'EdLevel', 'OrgSize', 'UndergradMajor',
       'Sexuality'],
      dtype='object')

Now we can compare each categorical variable against each other (using $\chi ^2$ tests of independence) to see if which categorical variables are independent of which (if any). We can only select variables that are indeendent of each other, or else we get multicollinearity in the model. So this is another way to eliminate features.

In [15]:
# this codeblock takes about 50s to run on my (slow) machine

chi_sq_tests = np.zeros((len(cat_vars.columns),len(cat_vars.columns)))

for i, rows in enumerate(cat_vars.columns):
    for j, cols in enumerate(cat_vars.columns):
        
        ctab_result = crosstab(cat_vars[rows], cat_vars[cols],
                               levels=(
                                   cat_vars[rows].unique(),
                                   cat_vars[cols].unique()))

        test_results = stats.chi2_contingency(ctab_result[1])
        chi_sq_tests[i][j] = test_results[1]  # load p-values into the grid
        amount_completed = 100*((len(cat_vars.columns)*i + j + 1)/len(cat_vars.columns)**2)
        print(" row:", i, 
              "col:", j, 
              "Progress: {:.2f}%".format(amount_completed),
              end="\r")  # progress

In [16]:
# redefine as a DataFrame, to have labels
chi_sq_tests = pd.DataFrame(chi_sq_tests, 
                            columns=cat_vars.columns, 
                            index=cat_vars.columns)
round(chi_sq_tests,2).to_csv('chi_sq_tests.csv')    # to save for later

In [46]:
indep_vars = chi_sq_tests.gt(0.05)           # show which cells in are greater than p=0.05, where null not rejected
print(indep_vars[indep_vars==True].count())  # looks like it was none of them!

Hobbyist          0
Trans             0
NEWOtherComms     0
NEWDevOps         0
CompFreq          0
SurveyLength      0
SurveyEase        0
NEWOffTopic       0
NEWOnboardGood    0
SOAccount         0
PurchaseWhat      0
JobSeek           0
NEWLearn          0
OpSys             0
NEWPurpleLink     0
JobSat            0
NEWDevOpsImpt     0
NEWEdImpt         0
NEWOvertime       0
MainBranch        0
SOComm            0
SOPartFreq        0
SOVisitFreq       0
WelcomeChange     0
Gender            0
Employment        0
EdLevel           0
OrgSize           0
UndergradMajor    0
Sexuality         0
dtype: int64


This means no two variables are independent of each other. So we could use exactly 1 categorical variable, but not another one, since we'd run into multicolinearity issues with two or more.

(discussion of quantitative variable selection)

Now we can take a look at our numerical variables, and see which ones are correlated with each other. If two variables are significantly correlated, we should not include both. The cutoff I chose is $r = 0.7$; correlations higher than that are deemed significant, with ones lower than that insignificant. So let's see what we get:

In [35]:
print(quant_vars.corr(numeric_only=True).gt(0.7))  # find where correlations are >0.6 in corr matrix

               Respondent    Age  Age1stCode  CompTotal  ConvertedComp  \
Respondent           True  False       False      False          False   
Age                 False   True       False      False          False   
Age1stCode          False  False        True      False          False   
CompTotal           False  False       False      False          False   
ConvertedComp       False  False       False      False           True   
WorkWeekHrs         False  False       False      False          False   
YearsCode           False   True       False      False          False   
YearsCodePro        False   True       False      False          False   

               WorkWeekHrs  YearsCode  YearsCodePro  
Respondent           False      False         False  
Age                  False       True          True  
Age1stCode           False      False         False  
CompTotal            False      False         False  
ConvertedComp        False      False         False  
WorkWeekH

(discussion of quatitative variable selection)

In [36]:
x_cols = []  # whatever we select
X = df[x_cols]
Y = df['Hobbyist']